# Proyecto final KDD

**Docente: Andrés Castillo**

**Autores**:
- Juan Camilo Sanchez Barreiro - 1527749
- Juan Felipe Orozco Escobar - 1426244

## Introdución

La estimación de los precios de la vivienda es esencial tanto para los propietarios como para los inversores, ya que ambos necesitan entender el valor de este activo inmobiliario. 
Para muchas personas, comprar una propiedad es una de las decisiones y compra más importantes en la vida. Además de la asequibilidad de una vivienda, otros factores, 
como la conveniencia del lugar y las perspectivas de inversión a largo plazo, también afectan
el proceso de toma de decisiones.
El mercado inmobiliario está expuesto a muchas fluctuaciones en los precios debido a las correlaciones existentes con muchas variables, algunas de las cuales no se pueden controlar
o incluso pueden ser desconocidas. Los precios de las viviendas pueden aumentar rápidamente (o en algunos casos, también bajan muy rápido).
Algunas aplicaciones para un banco son:
- Originación: Establecer el valor comercial del inmueble a financiar utilizado para la aprobación final.
- Retanqueo: Actualizar el valor comercial de garantía ya existente para aprobación de nuevos cupos de crédito.
- Monitoreo portafolio garantías: Valoración del portafolio de las garantías para cumplimiento normativo y para el análisis de riesgos del colateral.
- Normalización de cartera: Evaluar los préstamos existentes, evaluar los acuerdos de refinanciamiento Y daciones en pago.

## Objetivo:
El objetivo de este proyecto es que el estudiante aplique los temas vistos en la clase de Descubrimiento de Conocimiento para resolver un problema del mundo real, con datos reales. Lea cuidadosamente las instrucciones contenidas en el archivo "descripcion_prueba.pdf". Allí están los detalles iniciales de la prueba. Para este curso, se evaluará adicionalmente el desarrollo de los puntos de este notebook. 

Para resolver este problema, usted debe seguir las instrucciones especificas que se dan en cada parte del notebook. Al final, usted debe entregar una copia de este notebook junto con unas conclusiones finales que usted debe sacar y que deben estar contenidas al final del documento.

Cómo es usual, para empezar, debemos importar todas las librerías que vamos a necesitar. Asegurese de instalar la librería XGBoost para su sistema operativo y todas las demás, hasta que no encuentre ningún error al correr la siguiente celda:

In [1]:
# import the necessary packages

import pandas as pd
import numpy as np
import argparse
import glob
from pandas import DataFrame
import sklearn
from sklearn import tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_rows', 500)

ModuleNotFoundError: No module named 'xgboost'

Ahora definimos la funciones que nos ayudarán a evaluar el desempeño de nuestro modelo. Justo las mismas que se usaron en nuestro último ejercicio:

In [ ]:
# Calcula el mean_absolute_percentage_error. El vector y_true no puede tener 0
def mape(y_true, y_pred): 
    #y_true, y_pred = check_arrays(y_true, y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def plotScatterModel(aX, aY, bX, bY, model):
    paY = model.predict(aX)
    pbY = model.predict(bX)
    #Print the score on the train data
    print('R2')
    print({'Training': model.score(trainXX, trainY), 'Test': model.score(testXX, testY) })

    print('\nMAPE')
    print({'Training ': mape(trainY, paY), 'Test': mape(testY, pbY)})

    # declarando un objeto tipo Figura para desarrollar los subplots
    fig = plt.figure(figsize=(20, 8))

    ax = fig.add_subplot(1,2,1)
    plt.scatter(aY, paY, s = 0.1)
    plt.title('Correlation in training set')


    ax2 = fig.add_subplot(1,2,2)
    plt.scatter(bY, pbY, s = 0.1)
    plt.title('Correlation in test set')

Junto con este notebook, usted encontrará un conjunto de archivos con los datos de datos de entranamiento. 

!ls data

# Cargar los datos que usaremos

Como es un conjunto de datos pequeño podemos cargar todo en memoría y trabajar de esta manera. Solo usaremos los archivos entrenamiento_precios_vivienda.csv  y prueba_precios_vivienda.csv. Note que los datos del archivo prueba_precios_vivienda.csv no contienen la columna de los precios de la vivienda. La idea de este archivo, es que usted complete dicha columna con los predicciones resultantes de su modelo, y mediante un proceso de validación externo, la compañia calcula el desempeño de este. Esta es una práctica muy común en pruebas de este tipo.

In [ ]:
#Cargamos los datasets
low_memory=False
dataTrain = pd.read_csv('https://raw.githubusercontent.com/ijuanfe/uv-kdd_project/main/entrenamiento_precios_vivienda.csv', dtype=str)
dataTest = pd.read_csv('https://raw.githubusercontent.com/ijuanfe/uv-kdd_project/main/prueba_precios_vivienda.csv', dtype=str)

In [ ]:
# Asignar el campo id como índice del dataframe
dataTrain['id'] = pd.to_numeric(dataTrain['id'], errors='coerce')
dataTest['id'] = pd.to_numeric(dataTest['id'], errors='coerce')

dataTrain = dataTrain.set_index('id')
dataTest =  dataTest.set_index('id')
#dataTrain = dataTrain.drop(['id'], axis = 1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

dataTrain.head()

# Removemos todos los caracteres especiales y pasar todas las cadenas a mayúsculas


In [ ]:
import re

def stringCleanUpInPlace(dataX):
    attributes = dataX.columns

    for attribute in attributes:
        if type(dataX[attribute].values[0]) == str:
            dataX[[attribute]] = dataX[[attribute]].fillna('');
            dataX[attribute] = [re.sub('[^A-Za-z0-9.,+\-]+', '', row + '').upper() for row in dataX[attribute]]
    
    return dataX

            
stringCleanUpInPlace(dataTrain)
stringCleanUpInPlace(dataTest)

## Remove varias filas del conjunto de entranamiento que tienen las columnas del texto libre mal acotadas. Esto pasa
## pero al final la fila tiene el mismo número de columnas que las otras, entonces no se cual es realmente el avaluo.
## para evitar otros problemas, por ahora lo mejor es borrarlas
# print(dataTrain.shape)
dataTrain = dataTrain[dataTrain['alcantarillado_en_el_predio'].str.len() < 3]
dataTrain = dataTrain[dataTrain['numero_piso'].str.len() < 3]
dataTrain = dataTrain[dataTrain['habitaciones'].str.len() < 3]
dataTrain = dataTrain[dataTrain['estado_acabados_pisos'].str.len() < 15]
dataTrain = dataTrain[dataTrain['metodo_valuacion_1'].str.len() < 20]
dataTrain = dataTrain[dataTrain['estrato'].str.len() < 2]

### Dado que el Conjunto de datos de entrenamiento tiene algunos valores que no tienen relación con las variables,
### En esta parte, limpiamos esos datos que no tienen relación con la respectiva columna.

iluminacionValues = ['BUENO', 'REGULAR', 'MALO']#para que solamente queden los que tienen valor Bueno, Regular, Malo
dataTrain = dataTrain[dataTrain['iluminacion'].isin(iluminacionValues)]
estado_acabados_pisosValues = ['BUENO', 'REGULAR', 'SIN ACABADOS', 'MALO']
dataTrain = dataTrain[dataTrain['estado_acabados_pisos'].isin(estado_acabados_pisosValues)]
calidad_acabados_pisosValues = ['NORMAL', 'SENCILLO', 'LUJOSO', 'SIN ACABADOS']
dataTrain = dataTrain[dataTrain['calidad_acabados_pisos'].isin(calidad_acabados_pisosValues)]
motivoValues = ['GARANTA', 'CRDITOHIPOTECARIODEVIVIENDA', 'COMPRADECARTERA',
       'LEASINGHABITACIONAL', 'HIPOTECARIOVISTOBUENO', 'REMATES',
       'EMPLEADOS', 'DACINENPAGO', 'ACTUALIZACINDEGARANTAS',
       'LEASINGVISTOBUENO', 'CRDITOCOMERCIAL',
       'ACTUALIZACINDEGARANTASLEASING', 'COLOMEXTHIPOTECARIO', '0']
dataTrain = dataTrain[dataTrain['motivo'].isin(motivoValues)]
tipo_creditoValues = ['VIVIENDA', 'DIFERENTEDEVIVIENDA']
dataTrain = dataTrain[dataTrain['tipo_credito'].isin(tipo_creditoValues)]
fachadaValues = ['LADRILLOALAVISTA',
 'OTROS',
 'GRANIPLAST',
 'PAETEYPINTURA',
 'CONCRETOTEXTURADO',
 'FLOTANTE',]
dataTrain = dataTrain[dataTrain['fachada'].isin(fachadaValues)]
#data = dataTrain.copy()

print(dataTrain.shape)

In [ ]:
dataTrain['tipo_subsidio'].unique()

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#dataTrain = data.copy()
#dataTrain = dataTrain[dataTrain['estrato'].str.len() < 2]
# print(dataTrain.shape)
dataTrain.head(20)

# Clasificar los atributos según el tipo

Esta es una de las partes más importantes del proceso y es aquí donde necesitamos mas conocimiento del negocio. Me he basado en el diccionario de datos que entregaron y de la interpretación que pude hacer de los demás atributos. Como se puede ver, no he usado todas las variables. En el listado está solo la mitad de todas las diponibles.

Usualmente divido los atributos en numéricos, categóricos nominales y categóricos ordinales y texto no estructurado. Como no pienso usar los datos geoespaciales, voy a considerar por ahora la latitud y la longitud como variables numéricas.

**Tarea**

Usando la información disponible clasifique los atributos que usará para resolver el problema entre `numericColumns`, `factorColumns`, `levelColumns` y  `textColumns`

Las columnas objetivo son: `'valor_total_avaluo', 'valor_uvr', 'valor_avaluo_en_uvr'`
Aunque usaremos solo la primera realmente.

In [ ]:
from pandas.api.types import CategoricalDtype
# Nota: sacamos los siguientes atributos porque el datatest de pruebas los tiene todos en null
# - ['valor_area_privada', 'valor_area_libre', 'valor_area_construccion', 'valor_area_garaje', 'valor_area_deposito', 'valor_area_terreno', 'valor_area_otros']

numericColumns = ['area_privada', 'contadores_agua', 'unidades', 'fecha_aprobación', 'contadores_luz', 'accesorios',
                  'area_valorada', 'vetustez', 'bano_privado', 'bano_social', 'bano_servicio', 'cocina', 'estudio',
                  'balcon', 'terraza', 'patio_interior', 'jardin', 'zona_de_ropas', 'zona_verde_privada', 'local',
                  'oficina', 'bodega', 'numero_total_de_garajes', 'total_cupos_parquedaro', 'numero_total_depositos',
                  'area_privada', 'area_garaje', 'area_deposito', 'area_terreno', 'area_construccion', 'area_otros',
                  'Longitud', 'Latitud']

# Unordered Categorical (nominal)
factorColumns = ['objeto', 'motivo', 'proposito', 'tipo_avaluo', 'tipo_credito','tipo_subsidio',
                 'departamento_inmueble', 'municipio_inmueble',
                 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector',
                 'gas_en_el_sector', 'energia_en_el_sector', 'telefono_en_el_sector',
                 'vias_pavimentadas', 'sardineles_en_las_vias', 'andenes_en_las_vias',
                 'barrio_legal', 'topografia_sector', 'paradero', 'alumbrado', 'arborizacion',
                 'alamedas', 'ciclo_rutas', 'alcantarillado_en_el_predio', 'acueducto_en_el_predio',
                 'gas_en_el_predio', 'energia_en_el_predio', 'telefono_en_el_predio',
                 'tipo_inmueble', 'clase_inmueble', 'ocupante', 'sometido_a_propiedad_horizontal',
                 'altura_permitida', 'aislamiento_posterior', 'aislamiento_lateral', 'antejardin',
                 'indice_ocupacion', 'indice_construccion', 'predio_subdividido_fisicamente',
                 'condicion_ph', 'rph', 'porteria', 'citofono', 'bicicletero', 'piscina',
                 'tanque_de_agua', 'club_house', 'garaje_visitantes', 'teatrino', 'sauna', 'vigilancia_privada',
                 'administracion', 'estructura', 'ajustes_sismoresistentes', 'cubierta', 'fachada',
                 'estructura_reforzada', 'danos_previos', 'material_de_construccion', 'detalle_material',
                 'irregularidad_planta', 'irregularidad_altura', 'garaje_cubierto_1', 'garaje_doble_1',
                 'garaje_paralelo_1', 'garaje_servidumbre_1', 'tipo_deposito', 'area_libre',
                 'metodo_valuacion_1',
                 'metodo_valuacion_2', 'metodo_valuacion_3', 'metodo_valuacion_4','metodo_valuacion_5',
                 'metodo_valuacion_6', 'metodo_valuacion_7', 'metodo_valuacion_8', 'metodo_valuacion_9',
                 'garaje_cubierto_2', 'garaje_doble_2', 'garaje_paralelo_2', 'garaje_servidumbre_2',
                'garaje_cubierto_3','garaje_doble_3', 'garaje_paralelo_3', 'garaje_servidumbre_3',
                'garaje_cubierto_4','garaje_doble_4', 'garaje_paralelo_4', 'garaje_servidumbre_4', 'tipo_vigilancia','tipo_fachada',
                'garaje_cubierto_5','garaje_doble_5', 'garaje_paralelo_5', 'garaje_servidumbre_5'
                ] #excluimos uso_actual

# Ordered Categorical
levelColumns = ['condiciones_salubridad', 'iluminacion', 'ventilacion', 'estado_acabados_pisos', 'calidad_acabados_pisos',
                'estado_acabados_muros', 'calidad_acabados_muros', 'estado_acabados_techos', 'calidad_acabados_techos',
                'estado_acabados_madera', 'calidad_acabados_madera', 'estado_acabados_metal', 'calidad_acabados_metal',
                'estado_acabados_banos', 'calidad_acabados_banos', 'estado_acabados_cocina', 'calidad_acabados_cocina',
                'tipo_garaje', 'pisos_bodega','estrato']

textColumns = ['direccion_inmueble_informe', 'descripcion_general_sector', 'perspectivas_de_valorizacion', 'actualidad_edificadora',
               'comportamiento_oferta_demanda', 'descripcion_tipo_inmueble', 'descripcion_uso_inmueble', 'descripcion_clase_inmueble',
               'observaciones_generales_inmueble', 'area_actividad', 'observaciones_estructura', 'observaciones_dependencias',
               'numero_garaje_1', 'matricula_garaje_1', 'numero_garaje_2', 'matricula_garaje_2', 'numero_garaje_3', 'matricula_garaje_3',
               'numero_garaje_4', 'matricula_garaje_4', 'numero_garaje_5', 'matricula_garaje_5', 'numero_deposito_1', 'matricula_inmobiliaria_deposito_1',
               'numero_deposito_2', 'matricula_inmobiliaria_deposito_2', 'numero_deposito_3', 'matricula_inmobiliaria_deposito_3', 'numero_deposito_4',
               'matricula_inmobiliaria_deposito_4', 'numero_deposito_5', 'matricula_inmobiliaria_deposito_5', 'observaciones_generales_construccion',
               'concepto_del_metodo_3', 'concepto_del_metodo_2', 'concepto_del_metodo_1', 'concepto_del_metodo_4', 'concepto_del_metodo_5',
               'concepto_del_metodo_6', 'concepto_del_metodo_7', 'concepto_del_metodo_8', 'concepto_del_metodo_9', 'barrio', 'uso_principal_ph',
               'observaciones_altura_permitida', 'numero_piso', 'habitaciones','numero_de_edificios', 'estar_habitacion', 'cuarto_servicio',
               'observaciones_aislamiento_posterior', 'observaciones_aislamiento_lateral', 'observaciones_antejardin', 'closet', 'sala', 'comedor',
               'observaciones_indice_ocupacion', 'observaciones_indice_construccion']

target = ['valor_total_avaluo', 'valor_uvr', 'valor_avaluo_en_uvr']


def toDataTypesInPlace(dataX):
    # Convert to numeric attributes
    # Nothing to do in python. Numeric is the dafault
    realNumericColumns = []
    for col in numericColumns:
        if  col in dataX.columns:
            realNumericColumns.append(col)
            if type(dataX[col].values[0]) == str:
                dataX[col] = [re.sub(',', '.', row + '') for row in dataX[col]]
                dataX[col] = pd.to_numeric(dataX[col], errors='coerce')
  
    for col in target:
        if  col in dataX.columns:
            realNumericColumns.append(col)
            if type(dataX[col].values[0]) == str:
                dataX[col] = [re.sub(',', '.', row + '') for row in dataX[col]]
                dataX[col] = pd.to_numeric(dataX[col], errors='coerce')

    # loop to change each column to category type
    for col in factorColumns:
        if  col in dataX.columns:
            cat_type = CategoricalDtype(categories = None, ordered = False)
            dataX[col] = dataX[col].astype(dtype = cat_type)

    # Conver to levels / Ordinals
    for col in levelColumns:
      cat_type = CategoricalDtype(categories = None, ordered = True)
      dataX[col] = dataX[col].astype(dtype = cat_type)
    
    
    ### Especificamos exactamente el orden que debe llevar cada una de las variables ordinales
    col = 'demanda_interes'
    cat_type = CategoricalDtype(categories = [ 'NULA', 'DBIL', 'MEDIA', 'FUERTE'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'nivel_equipamiento_comercial'
    cat_type = CategoricalDtype(categories = [ 'REGULARMALO', 'ENPROYECTO', 'BUENO',  'MUYBUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estrato'
    cat_type = CategoricalDtype(categories = [ '0', '1', '2', '3', '4', '5', '6', '7'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'transporte'
    cat_type = CategoricalDtype(categories = ['MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'iluminacion'
    cat_type = CategoricalDtype(categories = ['MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'condiciones_salubridad'
    cat_type = CategoricalDtype(categories = ['MALAS', 'REGULARES', 'BUENAS'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'ventilacion'
    cat_type = CategoricalDtype(categories = ['MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estado_acabados_pisos'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO','REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_pisos'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'NORMAL', 'LUJOSO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estado_acabados_muros'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO','REGULAR','BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_muros'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'NORMAL', 'LUJOSO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)
    
    col = 'estado_acabados_techos'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_techos'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'NORMAL', 'LUJOSO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estado_acabados_madera'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_madera'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'NORMAL', 'LUJOSO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estado_acabados_metal'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_metal'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'NORMAL', 'LUJOSO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estado_acabados_banos'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_banos'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'NORMAL', 'LUJOSO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'estado_acabados_cocina'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS','MALO', 'REGULAR', 'BUENO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'calidad_acabados_cocina'
    cat_type = CategoricalDtype(categories = ['SIN ACABADOS', 'SENCILLO', 'SEMI-INTEGRAL', 'INTEGRAL'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'tipo_garaje'
    cat_type = CategoricalDtype(categories = ['NO TIENE', 'COMUNAL', 'PRIVADO', 'EXCLUSIVO'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    col = 'pisos_bodega'
    cat_type = CategoricalDtype(categories = ['0', '1', '2'], ordered = True)
    dataX[col] = dataX[col].astype(dtype = cat_type)

    # Delete the textColumns. I dont have time to deal with it now
    dataX = dataX[factorColumns + levelColumns + realNumericColumns]
    return dataX

dataTrain = toDataTypesInPlace(dataTrain);
dataTest = toDataTypesInPlace(dataTest);

# Miremos las variables nominales

Debemos hacer algo con el departamento, el municipio y el area_actividad. Talvez solo debemos considerar los principales valores de estas variables categóricas. He utilizado la visualización durante la curación de los datos. Esto me ha permitido saber que habían errores y así pude quitar todas la filas que estaban dañadas.

**Tarea**

Muestre diagramas tipo torta para una de las variables categoricas

In [ ]:

factorColumnsToGraph = ['topografia_sector', 'cubierta']
fig = plt.figure(figsize=(30, 60))

# Declarando las graficas de tipo Pie para Variables Categoricas
x = 1
for catAtt in factorColumnsToGraph:
    #Completar aqui
    ax = fig.add_subplot(1,2,x)
    dataTrain[catAtt].value_counts().plot(kind='pie', ax=ax, startangle=115, fontsize=12, rotatelabels = True, labeldistance=1)
    plt.title('Pie chart of '+catAtt)
    x = x + 1

plt.show()

# Miremos las variables numéricas (solo la variable objetivo en este caso)

In [ ]:
#dataTrain[dataTrain['valor_total_avaluo'] < 1e9]
dataTrain['valor_total_avaluo'].plot(kind='hist', rwidth=1)
plt.show()

Es claro que hay unos outlayer que harán que nuestro proceso de predicción sea dificil. Lo mejor es eliminarlos. Por ahora simplemente elimino los registros con valores por encima de 1e9. También lo mejor es eliminar los registros con avaluos muy bajos. No creo que algo en realidad valga menos de $100.000 por ejemplo 

In [ ]:
dataTrain[dataTrain['valor_total_avaluo'] < 1e9]['valor_total_avaluo'].plot(kind='hist', rwidth=1)
plt.show()
print(sum(dataTrain['valor_total_avaluo'] > 1e9))
print(sum(dataTrain['valor_total_avaluo'] <= 1000000))

**Tarea**

Removemos los ejemplos donde:
* dataTrain['valor_total_avaluo'] < 1e9
* dataTrain['valor_total_avaluo'] > 1000000

In [ ]:
dataTrain = dataTrain[dataTrain['valor_total_avaluo'] < 1e9 ]
dataTrain = dataTrain[dataTrain['valor_total_avaluo'] > 100000 ]
dataTrain.shape

In [ ]:
# Dado que la columna demanda_interes está duplicada, borramos una de las dos en ambos DataSets
dataTrain = dataTrain.loc[:,~dataTrain.columns.duplicated()]
dataTest = dataTest.loc[:,~dataTest.columns.duplicated()]

# Miremos las variables ordinales

**Tarea**

Muestre diagramas tipo torta para una de las variables ordinales

In [ ]:
levelColumnsToGraph = ['estrato', 'calidad_acabados_cocina']
fig = plt.figure(figsize=(30, 60))
x = 1
for catAtt in levelColumnsToGraph:
    ax = fig.add_subplot(1,2,x)
    dataTrain[catAtt].value_counts().plot(kind='pie', ax=ax, startangle=115, fontsize=12, rotatelabels = True, labeldistance=0.9)
    plt.title('Pie chart of '+catAtt)
    x = x + 1
plt.show()

# Separamos los datos de entramiento 

En nuestro conjunto de entramiento, tenemos los atributos descriptivos y la variable de interés(avaluo de la vivienda) en la misma matriz. A continuación debe separar los datos en `dataX` y `dataY`

**Tarea**

Separar los datos de manera que en dataY solo tenga el atributo target[0] y en dataX todos los atributos que no hagan parte de la lista target.

In [ ]:
dataY = dataTrain[target[0]]
dataX = dataTrain.drop(target, axis = 1)
## Eliminamos las columnas de target en DataSet
dataTest = dataTest.drop(target, axis = 1) 

# Convertir factores a dummy y levels a numeric

Ahora debemos convertir los atributos nominales y ordinales en variables numéricas. Adicionalmente debemos eleminar los datos con valores no definidos. 

In [ ]:
def dataToNumeric(dataIn, factorColumns, levelColumns):
    columns = dataIn.columns
    for catAtt in factorColumns:
        if catAtt in columns:
            dummies = pd.get_dummies(dataIn[catAtt], prefix = catAtt)
            dataIn = pd.concat([dataIn.drop(catAtt, axis = 1), dummies], axis = 1)
            

    for catAtt in levelColumns:
        if catAtt in columns:
            dataIn[catAtt] = dataIn[catAtt].cat.codes
    
    return dataIn


# Fill NAN in numerical attributes
def fillNaNWithMeanInPlace(dataX):
    foo = dataX.isnull().sum()
    index = 0
    for f in foo:
        if f > 0:
            colname = dataX.columns[index]
            print(colname)
            dataX[colname] = dataX[colname].fillna(dataX[colname].median())
            #else:
            #    dataX[colname] = dataX[colname].cat.add_categories('DESC')
            #    dataX[colname] = dataX[colname].fillna('DESC')
        index = index + 1
    return dataX

**Tarea**

Use las funciones definidas anteriormente para convertir todos los atributos a numéricos y para eliminar los datos faltantes.

In [ ]:
import warnings
data2 = dataX.copy()
        
data2 = dataToNumeric(data2, factorColumns, levelColumns)
dataTest = dataToNumeric(dataTest, factorColumns, levelColumns)

##Para manejar un error que salía en las columnas donde todos los valores son Nan
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    # foo = np.nanmean(x, axis=1)
    data2 = fillNaNWithMeanInPlace(data2)
    dataTest = fillNaNWithMeanInPlace(dataTest)

In [ ]:
data2.head()

**Tarea**

Elimine del conjunto de entrenamiento y test las columnas 'Latitud' y 'Longitud'.

Ayuda: Use la función drop con axis=1 del dataframe

In [ ]:
data2.drop(['Latitud', 'Longitud'], axis = 1, inplace=True)
dataTest.drop(['Latitud', 'Longitud'], axis = 1, inplace=True)

In [ ]:
print(data2.shape)
print(dataTest.shape)

Valores esperados:
    
```
(9310, 859)
(3175, 521)
```
    

# Primer experimento de datamining

Ajustamos la escala de las variables antes de comenzar a probar modelos. Y vamos a fijar el mismo conjunto de entrenamiento y test para todos los modelos de aqui hacia abajo. Para validar el desempeño de nuestro modelo en datos desconocidos, partimos nuestro conjunto de entranmiento en 2:
* Entranamiento: 75%
* Validación: 25%

Adicionalmente los datos x e y de entrenamiento y validación se deben estandarizar. 

**Tarea** 

Use la función `train_test_split` para partir los datos en entrenamiento y pruebsa.

In [ ]:
from sklearn.preprocessing import StandardScaler

# NN is sensitive to data scale. We must normilize
scaler = StandardScaler()  
scaler.fit(data2)
y = dataTrain[target[0]] / 1e9

# Se divide el set de datos en dos conjuntos train y test
trainX, testX, trainY, testY = train_test_split(data2, y, test_size = 0.25, train_size = 0.75)

# Don't cheat - fit only on training data
trainXX = scaler.transform(trainX)  
# apply same transformation to test data
testXX = scaler.transform(testX)

In [ ]:
trainX.shape

**Salida esperada**

```
(6982, 859)
```

# Probemos con un modelo que se pueda interpretar. 

La regresion lineal es el método más sencillo que podemos usar. Use la clase `LinearRegression` de `sklearn.linear_model` y verifique los resultado

In [ ]:
from sklearn.linear_model import LinearRegression

modelLM = LinearRegression()
modelLM.fit(trainXX, trainY);

plotScatterModel(trainXX, trainY, testXX, testY, modelLM)

Claramente un modelo lineal no resuelve el problema. Entonces si queremos poder interpretar el resultado vamos a tener que probar con otra cosa

# Decision Tree Regression.

**Tarea**

Una alternativa para obtener un modelo interpretable es usando árboles como modelo de regresión y preguntar por la importancia de las características. Use la clase`DecisionTreeRegressor` de `sklearn.tree`

Siguiendo el mismo esquema del modelo anterior implemente las intrucciones de la celda siguiente.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
modelDTR = DecisionTreeRegressor(max_depth=6)
modelDTR.fit(trainXX, trainY)

plotScatterModel(trainXX, trainY, testXX, testY, modelDTR)

Con estas líneas debería poder mostrar cual es la importancia de cada uno de los atributos del conjunto de datos. 

In [ ]:
attributes = trainX.columns[modelDTR.feature_importances_> 0]
importances = modelDTR.feature_importances_[modelDTR.feature_importances_> 0]
reportOfAttributes = pd.DataFrame({'attribute': attributes, 'importances': importances})
reportOfAttributes = reportOfAttributes.sort_values(by = 'importances',  ascending=False)
reportOfAttributes = reportOfAttributes.set_index('attribute')
reportOfAttributes.plot(kind='bar', figsize=(20, 5)).set_title('Feature importance Decision Tree Regression')
sum(modelDTR.feature_importances_> 0)

# XGBOOST

**Tarea**

Ahora entrene un modelo `XGBOOST`. Use estos parámetros: 

* objective='reg:squarederror
* colsample_bytree=0.2
* learning_rate=0.1
* max_depth=5
* alpha=5
* n_estimators=200

In [ ]:
xg_reg = xgboost.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.2, learning_rate=0.1, max_depth=5, alpha=5, n_estimators=200)
xg_reg.fit(trainXX, trainY)

plotScatterModel(trainXX, trainY, testXX, testY, xg_reg)

**Tarea**

Nuevamente, muestre la importancia de cada uno de los atributos según el algoritmo XGBoost

In [ ]:
attributes = trainX.columns[xg_reg.feature_importances_> 0]
importances = xg_reg.feature_importances_[xg_reg.feature_importances_> 0]
reportOfAttributes = pd.DataFrame({'attribute': attributes, 'importances': importances})
reportOfAttributes = reportOfAttributes.sort_values(by = 'importances',  ascending=False)
reportOfAttributes = reportOfAttributes.set_index('attribute')
reportOfAttributes.plot(kind='bar', figsize=(20, 5)).set_title('Feature importance XGBOOST')
sum(xg_reg.feature_importances_> 0)

# ADABoost

Entrene un modelo ADABoost

In [ ]:
# ADABoost
from sklearn.ensemble import AdaBoostRegressor
modelADA = AdaBoostRegressor(n_estimators=200, learning_rate=0.1)
modelADA.fit(trainXX, trainY)

plotScatterModel(trainXX, trainY, testXX, testY, modelADA)

**Tarea**

Muestre la importancia de los atributos para el modelo ADABoost

In [ ]:
attributes = trainX.columns[modelADA.feature_importances_> 0]
importances = modelADA.feature_importances_[modelADA.feature_importances_> 0]
reportOfAttributes = pd.DataFrame({'attribute': attributes, 'importances': importances})
reportOfAttributes = reportOfAttributes.sort_values(by = 'importances',  ascending=False)
reportOfAttributes = reportOfAttributes.set_index('attribute')
reportOfAttributes.plot(kind='bar', figsize=(20, 5)).set_title('Feature importance ADABoost')
sum(modelADA.feature_importances_> 0)

# Random Forest regressor

**Tarea**

Entrene un modelo de regresión Random Forest. Use la clase `RandomForestRegressor` de `sklearn.ensemble`

In [ ]:
# RadomForest
from sklearn.ensemble import RandomForestRegressor
modelRFR = RandomForestRegressor(max_depth=5, n_estimators=200)
modelRFR.fit(trainXX, trainY)

plotScatterModel(trainXX, trainY, testXX, testY, modelRFR)

**Tarea**

Muestre la importancia de los atributos para el modelo random forest.

In [ ]:
attributes = trainX.columns[modelRFR.feature_importances_> 0]
importances = modelRFR.feature_importances_[modelRFR.feature_importances_> 0]
reportOfAttributes = pd.DataFrame({'attribute': attributes, 'importances': importances})
reportOfAttributes = reportOfAttributes.sort_values(by = 'importances',  ascending=False)
reportOfAttributes = reportOfAttributes.set_index('attribute')
reportOfAttributes.plot(kind='bar', figsize=(20, 5)).set_title('Feature importance Random Forest regressor')
sum(modelRFR.feature_importances_> 0)

# Red neuronal

**Tarea**

Entrene una red neuronal para este problema. Busque de manera libre los parámetros que mejor le funcionen.

In [ ]:
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import normalize
from keras.optimizers import Adamax

# Completar
trainXX = normalize(trainXX, axis=1)
testXX = normalize(testXX, axis=1)

neural_network_model = MLPRegressor(batch_size=32, hidden_layer_sizes=1)
neural_network_model.fit(trainXX, trainY)

# Print report
plotScatterModel(trainXX, trainY, testXX, testY, neural_network_model)

# Selección de atributos

**Tarea**

Ahora, para hacer que el modelo XGBoost se demore menos entrenando, use solo los mejores atributos del modelo ADABoost, como atributos de entrada al modelo XGBoost. 

Para esto puede seleccionar las columnas de la siguiente manera:

```py
data3 = data2.loc[:, data2.columns[modelAda.feature_importances_> 0].values]
```

In [ ]:
# Usemos los mejores atributos del modelo ADABoost para entrenar el resto de modelos de aqui en adelante
data2.shape
data3 = data2.loc[:, data2.columns[modelADA.feature_importances_ >= 0.000685].values]
data3.head()

In [ ]:
scaler = StandardScaler()
scaler.fit(data3)
y = dataTrain[target[0]] / 1e9

# Se divide el set de datos en dos conjuntos train y test
trainX, testX, trainY, testY = train_test_split(data3, y, test_size = 0.25)

# Don't cheat - fit only on training data
trainXX = scaler.transform(trainX)
# apply same transformation to test data
testXX = scaler.transform(testX)

**Tarea**

Entrene un nuevo modelo XGBoost con el nuevo conjunto de datos. Compare los resultados y el tiempo requerido para el entrenamiento.

In [ ]:
xg_reg2 = xgboost.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.2, learning_rate=0.1, max_depth=5, alpha=5, n_estimators=200)
xg_reg2.fit(trainXX, trainY)

plotScatterModel(trainXX, trainY, testXX, testY, xg_reg2)

In [ ]:
### Esto lo hicimos con el proposito de saber cuales son los features mas importantes para entrenar el próximo
### módelo de XGBOOST. Tomamos hasta Feature: garaje_doble_2_0 	Importance: 0.000685
attributes = data2.columns[modelADA.feature_importances_> 0]
importances = modelADA.feature_importances_[modelADA.feature_importances_> 0]
reportOfAttributes = pd.DataFrame({'attribute': attributes, 'importances': importances})
reportOfAttributes = reportOfAttributes.sort_values(by = 'importances',  ascending=False)
reportOfAttributes = reportOfAttributes.set_index('attribute')
reportOfAttributes

# Predecir

Use el mejor modelo obtenido para predecir los precios de todas la viviendas del conjunto de `dataTest`. Guarde el resultado en un archivo 'predicciones.csv'

Para entregar, deberá poner en una carpeta este notebook, junto con el archivo de las las predicciones para el conjunto de test

In [ ]:
## Seleccionamos de DataTest las variables que tienen mayor incidencia en este
## problema según AdaBoost
data4 = dataTest.loc[:, data2.columns[modelADA.feature_importances_ >= 0.000685].values]
data4.head()

In [ ]:
## Realizamos la predicción utilizando el segundo modelo XGBOOST
predictions = xg_reg2.predict(data4.values) * 1e9
predictions

In [ ]:
## Formateamos la salida de las predicciones en un nuevo DataFrame
output = pd.DataFrame(data=predictions, index=dataTest.index)
output.head()

In [ ]:
## Escribimos el DataFrame 'output' en un archivo CSV
output.to_csv('base_evaluada.csv')